In [28]:
import sys
sys.path.append('/scratch/ty296/CT_MPS_mini')
from read_hdf5_func import von_neumann_entropy_sv
import h5py
import numpy as np
import os

# Read the HDF5 file
filename = '/scratch/ty296/hdf5_data/p_ctrl0.0/46376498_a0_L10.h5'


In [29]:
print(sys.path)
print(os.getcwd())

['/projects/community/python/3.9.6/gc563/lib/python39.zip', '/projects/community/python/3.9.6/gc563/lib/python3.9', '/projects/community/python/3.9.6/gc563/lib/python3.9/lib-dynload', '', '/home/ty296/.local/lib/python3.9/site-packages', '/projects/community/python/3.9.6/gc563/lib/python3.9/site-packages', '/scratch/ty296/CT_MPS_mini', '/scratch/ty296/CT_MPS_mini', '/scratch/ty296/CT_MPS_mini', '/scratch/ty296/CT_MPS_mini']
/scratch/ty296


In [30]:
p_proj_list = np.linspace(0.0,1.0,20)
p_target = p_proj_list[3]
print(p_target)

0.15789473684210525


In [43]:
maxbonds = []
counter = 0
with h5py.File(filename, 'r') as f:
    print("Keys in HDF5 file:")
    keys_list = list(f.keys())
    for key in keys_list:
        print(key)
        metadata = f[key]
        print(metadata.keys())
        result_1 = metadata['result_3']
        print(type(result_1))
        print(result_1.keys())
        print(result_1['seed'][()])
        print(result_1['max_bond'][()])
        print(result_1['realization'][()])
        print(result_1['args'].keys())
    #     L = f[key].attrs['L']
    #     p_proj = f[key].attrs['p_proj']
    #     args = dict(f[key].attrs)
    #     maxbond = args['maxbond']
    #     # print(maxbond)
    #     if L == 24 and p_proj == p_target:
    #         counter += 1
    #         # print(f[key].attrs['L'])
    #         # print(f"  {L}: {len(f[key][()])}: {f[key][()]}")
    #         print(maxbond)
    #         maxbonds.append(maxbond)
    # print(f"Found {counter} datasets")

# need to store seed and maxbonddim

Keys in HDF5 file:
metadata
<KeysViewHDF5 ['result_1', 'result_2', 'result_3', 'result_4', 'result_5']>
<class 'h5py._hl.group.Group'>
<KeysViewHDF5 ['O', 'args', 'max_bond', 'p_ctrl', 'p_proj', 'realization', 'seed']>
8084
4
1
<KeysViewHDF5 ['L', 'ancilla', 'cutoff', 'job_id', 'maxdim', 'n_chunk_realizations', 'output_dir', 'p_fixed_name', 'p_fixed_value', 'p_range', 'random', 'store_sv']>
singular_values
<KeysViewHDF5 ['result_1', 'result_2', 'result_3', 'result_4', 'result_5']>
<class 'h5py._hl.dataset.Dataset'>


AttributeError: 'Dataset' object has no attribute 'keys'

8388608